In [76]:
import re
def parse_row(row):
    category = int(row.split(";")[0])
    row = ";".join(row.split(";")[1:])
    authors_indexes = row.split("http")[0][:-1]
    authors = authors_indexes.split('""')[0][1:].split(".,")
    authors = [(author+".").strip() for author in authors if len(author)>0]
    indexes = authors_indexes.split('""')[1]
    indexes = re.sub(r'[^\w\s]',' ',indexes).split()
    indexes = list(map(int, indexes))
    url = "http" + row.split("http")[1].split('""')[0] 
    keywords = '""'.join(row.split("http")[1].split('""')[1:])[1:].split(";")
    keywords = [re.sub(r'[^\w\s]',' ',keyword) for keyword in keywords]
    keywords = [" ".join(keyword.split()) for keyword in keywords]
    keywords = [keyword for keyword in keywords if len(keyword)>0]
    return [category, authors, indexes, url, keywords]

In [77]:
data = []
for year in ["2011"]:
    with open('authors_keywords_{year}.csv'.format(year = year), 'r') as file:
        for index, row in enumerate(file):
            if index ==0:
                continue
            category, authors, indexes, url, keywords = parse_row(row)
            assert(len(authors) == len(indexes))
            out = {"year":year,
                   "category": category,
                   "url": url,
                   "keywords": "; ".join(keywords),
                   "authors": [{"id": idd, "author": author} for idd, author in zip(indexes,authors)]}
            data.append(out)

In [78]:
import json

## Write json
with open('new_data.json', 'w') as outfile:
    json.dump(data, outfile)

## Read json
with open('new_data.json') as f:
    new_data = json.load(f)

new_data[0]

{'year': '2011',
 'category': 0,
 'url': 'https://www.scopus.com/inward/record.uri?eid=2-s2.0-79961238208&partnerID=40&md5=f70f5d965f80944d48a76f2d5327ab3d',
 'keywords': 'Anomalous diffusion; Fractal operator; Random',
 'authors': [{'id': 56802827200, 'author': 'Gromov E.'},
  {'id': 8284266700, 'author': 'Logvinova K.'},
  {'id': 37079439900, 'author': 'Morozov V.'},
  {'id': 6603597680, 'author': 'Tyutin V.'}]}

In [79]:
##TEST
for text_row, answer in [('0;"Gromov E., Logvinova K., Morozov V., Tyutin V.,""56802827200";8284266700;37079439900;6603597680;",https://www.scopus.com/inward/record.uri?eid=2-s2.0-79961238208&partnerID=40&md5=f70f5d965f80944d48a76f2d5327ab3d"",""Anomalous diffusion"; Fractal operator;" Random"",";;;;;;;;;;;;;;;;;;;;;;;;;;;;;',
                        [0,
                         ["Gromov E.", "Logvinova K.", "Morozov V.", "Tyutin V."],
                         [56802827200, 8284266700, 37079439900, 6603597680],
                         "https://www.scopus.com/inward/record.uri?eid=2-s2.0-79961238208&partnerID=40&md5=f70f5d965f80944d48a76f2d5327ab3d",
                         ["Anomalous diffusion", "Fractal operator", "Random"]])]:
    assert(parse_row(text_row)==answer), text_row